Notebook pour m'aider à filtrer les features, leur représentativité etc et juger si elles sont catégorielels ou non

In [2]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import norm
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
# !pip install openml
import openml
# !pip install cardinal
# !pip install scikit-learn==0.20.4
# !pip install sklearn.cluster._k_means_fast
from cardinal.uncertainty import MarginSampler
from cardinal.random import RandomSampler
from cardinal.zhdanov2019 import TwoStepKMeansSampler
from cardinal.plotting import plot_confidence_interval
import tqdm


import os
import tensorflow as tf
from sklearn.model_selection import KFold, StratifiedKFold
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import clone_model

In [3]:
np.random.seed(123)

# Détermination manuelle des features catégorielles OpenML

In [4]:
dataset_ids = [1471, 1502, 40922, 41162, 43551, 1461, 1590, 41138, 42395, 42803, 43439]  

In [55]:
dataset = openml.datasets.get_dataset(42803)
X, y, cat_indicator, names = dataset.get_data(dataset_format='dataframe', target=dataset.default_target_attribute)  #dataframe / array
cat_indicator = np.asarray(cat_indicator)

start_index = 0
# print("Nb features : ", len(X.columns))
# print()

i=0
for feat_name, cat_bool in zip(X.columns[start_index:], cat_indicator[start_index:]):
    # if len(np.unique(X[feat_name])) <500:
    # print(cat_bool, feat_name, "            ", X[feat_name][0])
    # print(cat_bool, feat_name,  "       ",len(np.unique(X[feat_name])), np.unique(X[feat_name]) )#   np.unique(X[feat_name]) '        SAMPLE:',X[feat_name][:1])
        # print()


    if feat_name == "Date":
        print(i)
    #     # print(len(np.unique(X["LSOA_of_Accident_Location"])))
    #     # print(X["LSOA_of_Accident_Location"])
    #     # if float("nan") in X[feat_name].values:
    #     #     print("None found")

    # print(X[feat_name].values)


    # if 11.0 in X[feat_name].values:
    #     print(i, feat_name,"       ", list(X[feat_name].values).count(11.0))
    i+=1


# X

#features incertaines: 

# idx feat problématiques/inanalysables pour le dataset de voitures :  28, 30,32, 50

# todo cat : Vehicle_Reference_df_res, Vehicle_Type, Towing_and_Articulation, Vehicle_Manoeuvre, Vehicle_Location-Restricted_Lane, Junction_Location, Skidding_and_Overturning, Hit_Object_in_Carriageway, Vehicle_Leaving_Carriageway, Hit_Object_off_Carriageway, 1st_Point_of_Impact, Was_Vehicle_Left_Hand_Drive?, Journey_Purpose_of_Driver

28


In [43]:
feat_name = 'Accident_Index'
print(len(np.unique(X[feat_name])))
print(np.unique(X[feat_name]))


#add cat VehYear, VehicleAge

140056
['201501BS70001' '201501BS70002' '201501BS70004' ... '2015984140215'
 '2015984140515' '2015984141415']


In [40]:
to_cat = ['Vehicle_Reference_df_res', 'Vehicle_Type', 'Towing_and_Articulation', 'Vehicle_Manoeuvre', 'Restricted_Lane', 'Junction_Location', 'Skidding_and_Overturning', 'Hit_Object_in_Carriageway', 'Vehicle_Leaving_Carriageway', 'Hit_Object_off_Carriageway',
'Vehicle_Leaving_Carriageway', 'Hit_Object_off_Carriageway', '1st_Point_of_Impact', 'Was_Vehicle_Left_Hand_Drive?', 'Journey_Purpose_of_Driver', 'Age_Band_of_Driver', 'Propulsion_Code','Driver_Home_Area_Type', 'Police_Force', 'Accident_Severity', 'Number_of_Vehicles', 'Number_of_Casualties',
'Day_of_Week', 'Local_Authority_(District)', '1st_Road_Class', 'Road_Type', 'Speed_limit', 'Junction_Detail', 'Junction_Control', '2nd_Road_Class','Pedestrian_Crossing-Human_Control', 'Pedestrian_Crossing-Physical_Facilities', 'Light_Conditions' , 'Weather_Conditions' , 'Road_Surface_Conditions', 
'Urban_or_Rural_Area', 'Did_Police_Officer_Attend_Scene_of_Accident','Vehicle_Reference_df', 'Casualty_Reference', 'Casualty_Class', 'Sex_of_Casualty', 'Age_Band_of_Casualty', 'Casualty_Severity', 'Pedestrian_Location','Pedestrian_Movement', 'Car_Passenger', 'Bus_or_Coach_Passenger', 
'Pedestrian_Road_Maintenance_Worker', 'Casualty_Type', 'Casualty_Home_Area_Type', 'Casualty_IMD_Decile']
cat_indicator_TRUE = cat_indicator

for index, feat_name in enumerate(X.columns):
    if feat_name in to_cat:
        cat_indicator_TRUE[index] = True #"CAT"
    else:
        # print ("hop")
        cat_indicator_TRUE[index] = False#"NUM"

print(cat_indicator_TRUE)

[False  True  True  True  True False  True  True  True  True  True  True
  True  True False  True False  True False  True False False False False
  True  True  True  True False  True False  True False  True False  True
  True  True  True  True False  True  True  True  True  True False False
  True  True False  True  True  True  True False  True  True  True  True
  True  True  True  True  True  True]


### Examen features incertaines

In [149]:
#features incertaines: education-num, hours-per-week
feat = "V15"
print(len(np.unique(X[feat])))
print(np.unique(X[feat]))

41
[  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.
  14.  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.
  28.  29.  30.  32.  35.  37.  38.  40.  41.  51.  55.  58. 275.]


### 43439

Drop: PatientId, AppointmentId

Cat: Gender, ScheduledDay, AppointmentDay Neighbourhood, Scholarship, Hipertension, Diabetes, Alcoholism, Handcap, SMS_received
Numeric: Age


### 43551

Add cat : is_male, logical_score, verbal_score

### 1461

Add cat : V1, V13, V15  

### 1590

Add cat : education-num, hours-per-week

### 41138

Suspicieux catégorie ? "eg_000" comme nombreux autres